# Better way 20. 동적 기본 인수를 지정하려면 None과 docstring을 사용하자

In [1]:
from time import sleep
from datetime import datetime
import json

* 키워드 인수의 기본값으로 non-static 비정적 타입을 사용해야 할 때도 있다.
* 예) 이벤트 발생 시각을 메시지에 포함하고 싶을 때 기본 인수로 함수를 호출 _(나쁜 예시)_

In [2]:
def log(message, when=datetime.now()):
    print('%s: %s' % (when, message))
    
log('Hi there!')
sleep(0.1)
log('Hi again!')

2019-05-19 15:11:26.683586: Hi there!
2019-05-19 15:11:26.683586: Hi again!


* `datetime.now` 는 함수를 정의할 때 딱 한번 실행됨
  * = 모듈이 로드될 때

* 결과가 기대한 대로 나오게 하려면 기본값을 None 으로 설정하고 docstring (문서화 문자열) 으로 실제 동작을 문서화하는게 관례
  * Better way 49. 모든 함수, 클래스, 모듈에 docstring 을 작성하자

In [3]:
def log(message, when=None):
    """Log a message with a timestamp
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))
    
log('Hi there!')
sleep(0.1)
log('Hi again!')

2019-05-19 15:11:26.796386: Hi there!
2019-05-19 15:11:26.900804: Hi again!


* 기본 인수로 None 을 사용하는 방법은 인수가 mutable (수정 가능) 할 때 특히 중요
* 예시) Json 데이터로 인코드된 값을 로드할 때
  * 데이터 디코딩이 실패하면 기본값으로 빈 딕셔너리를 반환 (나쁜 예시)

In [4]:
def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [5]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1

print('Foo:', foo)
print('Var:', bar)

Foo: {'stuff': 5, 'meep': 1}
Var: {'stuff': 5, 'meep': 1}


In [6]:
assert foo is bar

* 기본값을 None 으로 설정하고 docstring 에 동작을 문서화하자

In [7]:
def decode(data, default=None):
    """Load JSON data from a string
    
    Args:
        data: JSON data to decode
        default: Value to return if decoding fails.
            Defaults to an empty dictionary
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

In [8]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1

print('Foo:', foo)
print('Var:', bar)

Foo: {'stuff': 5}
Var: {'meep': 1}


### 핵심 정리
* 기본 인수는 모듈 로드 시점에 함수 정의 과정에서 딱 한번만 평가된다. 그래서 ({}나 [] 같은) 동적 값에는 이상하게 동작하는 원인이 되기도 한다.
* 값이 동적인 키워드 인수에는 기본값으로 None을 사용하자. 그러고 나서 함수의 docstring에 실제 기본 동작을 문서화하자.

# Better Way 21. 키워드 전용 인수로 명료성을 강요하자

* 키워드로 인수를 넘기는 방법은 파이썬의 강력한 기능
* 키워드 인수의 유언성 --> 쓰임새가 분명하게 코드 작성 가능
* 예시) 어떤 숫자를 다른 숫자로 나눌 때 (아쉬운 예시)
  * ZeroDivisionError 예외를 무시하고 무한대값을 반환하거나
  * OverflowError 예외를 무시하고 0을 반환하고 싶음

In [9]:
def safe_division(number, divisor, ignore_overflow, ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [10]:
result = safe_division(1, 10**500, True, False)
print(result)

0.0


In [11]:
result = safe_division(1, 0, False, True)
print(result)

inf


* argument 위치가 헷갈릴 수 있어서 찾기 어려운 버그가 발생할 수 있음
* 키워드 인수를 사용하자 (아쉬움)

In [12]:
def safe_division_b(number, divisor, ignore_overflow=False, ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [13]:
result = safe_division_b(1, 10**500, ignore_overflow=True)
print(result)

0.0


In [14]:
result = safe_division_b(1, 0, ignore_zero_division=True)
print(result)

inf


* 함수를 호출하는 쪽에 키워드 인수로 의도를 명확하게 드러내라고 강요할 방법이 없음
* 여전히 위치 인수를 사용하는 방식으로 호출할 수 있다.

In [15]:
result = safe_division_b(1, 10**500, True, False)
print(result)

0.0


* 복잡한 함수를 작성할 때에는 호출하는 쪽에서 의도를 명확히 드러내도록 요구하는 게 낫다.
* Python3 에서는 키워드 전용 인수 keyword-only-argument 를 사용

In [16]:
def safe_division_c(number, divisor, *, ignore_overflow=False, ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [17]:
result = safe_division_c(1, 10**500, True, False)
print(result)

TypeError: safe_division_c() takes 2 positional arguments but 4 were given

In [18]:
result = safe_division_c(1, 10**500, ignore_overflow=True)
print(result)

0.0


In [19]:
try:
    safe_division_c(1, 0)
except ZeroDivisionError:
    pass

### Python 2의 키워드 전용 인수

* Python 2 에는 키워드 전용 인수를 지정하는 명시적 문법이 없음
* 대신 인수 리스트에 `**` 키워드 연산자를 사용해 올바르지 않은 함수 호출을 할 때 TypeError 를 일으키는 방법으로 같은 동작을 할 수 있음
* `**` 의 차이
  * 키워드 인수를 몇개든 (정의하지 않았을 때 조차도) 받을 수 있음
  * 나머지는 비슷

In [20]:
# 주의: Python 3 로 실행되고 있음
def print_args(*args, **kwargs):
    print('Positional:', args)
    print('Keyword:', kwargs)

In [21]:
print_args(1, 2, foo='bar', stuff='meep')

Positional: (1, 2)
Keyword: {'foo': 'bar', 'stuff': 'meep'}


In [22]:
def safe_division_d(number, divisor, **kwargs):
    ignore_overflow = kwargs.pop('ignore_overflow', False)
    ignore_zero_div = kwargs.pop('ignore_zero_division', False)
    
    if kwargs:
        raise TypeError('Unexpected **kwargs: %r' % kwargs)
    
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_div:
            return float('inf')
        else:
            raise

In [23]:
result = safe_division_d(1, 10)
print(result)

0.1


In [24]:
result = safe_division_d(1, 0, ignore_zero_division=True)
print(result)

inf


In [25]:
result = safe_division_d(1, 10**500, ignore_overflow=True)
print(result)

0.0


* 키워드 전용 인수를 위치로 넘기려고 하면 제대로 동작하지 않음

In [26]:
result = safe_division_d(1, 0, False, True)
print(result)

TypeError: safe_division_d() takes 2 positional arguments but 4 were given

* 원하지 않는 키워드 인수를 넘겨도 제대로 동작하지 않음

In [27]:
result = safe_division_d(1, 0, unexpected=True)
print(result)

TypeError: Unexpected **kwargs: {'unexpected': True}

### 핵심 정리
* 키워드 인수는 함수 호출의 의도를 더 명확하게 해준다.
* 특히 bool 플래그를 여러개 받는 함수처럼 헷갈리기 쉬운 함수를 호출할 때 키워드 인수를 넘기게 하려면 키워드 전용 인수를 사용하자.
* 파이썬 3는 함수의 키워드 전용 인수 문법을 명시적으로 지원한다.
* 파이썬 2에서는 `**kwargs` 를 사용하고 TypeError 예외를 직접 일의는 방법으로 함수의 키워드 전용 인수를 흉내낼 수 있다.